In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go 

### Reading In Tables
    Keys built in

In [ ]:
admin_df = pd.read_csv('../Data/Mart2/admin.csv')
other_df = pd.read_csv('../Data/Mart2/other.csv')
locat_df = pd.read_csv('../Data/Mart2/location_compiled.csv')
admin_df.drop('Unnamed: 0', axis=1, inplace=True)
other_df.drop('Unnamed: 0', axis=1, inplace=True)
locat_df.drop('Unnamed: 0', axis=1, inplace=True)

# dropping duplicates throughout
admin_df.drop_duplicates('Incident_Key', inplace=True) # minimal loss
other_df.drop_duplicates('Incident_Key', inplace=True) # minimal loss
locat_df.drop_duplicates(inplace=True) # duplicated Incident_Key include multiple primary locations

In [ ]:
# census data for 2019 population estimates, source:
# https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/
census = pd.read_excel('../Data/Mart2/tn_county_pop.xlsx',sheet_name='Sheet1')

In [ ]:
census.head()

In [ ]:
# missing city
admin_df.Address.isnull().value_counts()

### Decided to run county for time
Other includes county without needing to parce inconsistent/broken address field from Admin

In [ ]:
def mult_pri_local(key, list_of_locations):
    # number of times to duplicate entries, but not the weights (for Rates)
    n = len(list_of_locations)
    for i in range(n):
        if i > 0:
            mart2_dict['Incident_Key'].append(key)
        mart2_dict['Incident_Date'].append(dt)
        mart2_dict['County'].append(county)
        if i == 0:
            mart2_dict['Weight'].append(weight)
        else:
            mart2_dict['Weight'].append(0)
        mart2_dict['Agency_Name'].append(agency_name)
        mart2_dict['Primary_Location'].append(list_of_locations[i])

    return

mart2_dict = {
    'Incident_Key':list(admin_df.Incident_Key),
    'Incident_Date':[],
    'County':[],
    'Weight':[],
    'Agency_Name':[],
    'Primary_Location':[],
}

for key in list(admin_df.Incident_Key):
    oth = other_df.loc[other_df.Incident_Key == key]
    # pull county from other_df
    county = oth.County.values[0]
    # pull weight
    weight = oth['Number of Offenses in Incident'].values[0]
    # pull Agency Name
    agency_name = oth['Agency Name'].values[0]
    locat = locat_df.loc[locat_df.Incident_Key == key]
    # pull date
    dt = locat['Incident Date'].values[0]
    # pull Pri Local(s)
    pri_loc = locat['Primary Location Type'].unique().tolist()
    
    # append to dictionary
    mult_pri_local(key,pri_loc)



In [ ]:
locat_df.loc[locat_df.Incident_Key == 'TN0410000<>20010101-133<>01/01/20']['Primary Location Type'].unique().tolist()

In [ ]:
# creating table
mart2_df = pd.DataFrame(mart2_dict)
mart2_df.head()

In [ ]:
mart2_df.to_csv('../Tables/Mart2/Mart2_Table1.csv')

In [ ]:
by_county = mart2_df.groupby(['County']).sum()
by_county.reset_index(drop=False, inplace=True)
by_county.rename({'Weight':'DV_Incident_Frequency'}, axis=1, inplace=True)
by_county['Pop_Est_2019'] = census['2019 Estimate']
by_county['Offense Rate Per 1k'] = round(by_county.DV_Incident_Frequency / by_county.Pop_Est_2019 * 1000, 4)
by_county['Offense Rate Per 10k'] = round(by_county.DV_Incident_Frequency / by_county.Pop_Est_2019 * 10000, 4)
by_county['Offense Rate Per 100k'] = round(by_county.DV_Incident_Frequency / by_county.Pop_Est_2019 * 100000, 4)
by_county.head()

In [ ]:
by_county_count = mart2_df.groupby(['County']).count()
by_county_count.reset_index(drop=False, inplace=True)
by_county_count.rename({'Weight':'DV_Incident_Frequency'}, axis=1, inplace=True)
by_county_count.drop(['Incident_Key','Incident_Date', 'Agency_Name', 'Primary_Location'], axis=1, inplace=True)
by_county_count.head()


In [ ]:
by_county['DV_Incident_Frequency'] = by_county_count.DV_Incident_Frequency
by_county['Incident Rate Per 1k'] = round(by_county.DV_Incident_Frequency / by_county.Pop_Est_2019 * 1000, 4)
by_county['Incident Rate Per 10k'] = round(by_county.DV_Incident_Frequency / by_county.Pop_Est_2019 * 10000, 4)
by_county['Incident Rate Per 100k'] = round(by_county.DV_Incident_Frequency / by_county.Pop_Est_2019 * 100000, 4)
by_county.head()

In [ ]:
by_county.to_csv('../Tables/Mart2/DV_Rates_by_County.csv')

In [ ]:
by_agency = mart2_df.groupby(['Agency_Name']).count()
by_agency.reset_index(drop=False, inplace=True)
by_agency.drop(['Incident_Key','Incident_Date','County', 'Primary_Location'], axis=1, inplace=True)
by_agency.rename({'Weight':'DV_Incident_Frequency'}, axis=1, inplace=True)
by_agency.head()

In [ ]:
def city_or_county(x):
    if 'County' in x:
        return 'County'
    elif 'Air' in x:
        return 'Airport'
    elif ' College' in x or 'University' in x:
        return 'College/Univerity'
    else:
        return 'City'

by_agency['Agency_Type'] = by_agency.Agency_Name.apply(city_or_county)
by_agency = by_agency[['Agency_Name','Agency_Type','DV_Incident_Frequency']]
by_agency.head()

In [ ]:
# pie charts with top 10 + 11th other category for both by_agency and by_county
batotal = by_agency.DV_Incident_Frequency.sum() # 60149
bctotal = by_county.DV_Incident_Frequency.sum() # 60149
# top 10s
baT10 = by_agency.sort_values('DV_Incident_Frequency', ascending=False)[:10]
baT10.index = np.arange(1,11)
bcT10 = by_county.sort_values('DV_Incident_Frequency', ascending=False)[:10]
bcT10.index = np.arange(1,11)
# trimming bcT10's featuers
bcT10 = bcT10[['County','DV_Incident_Frequency','Pop_Est_2019',]]
# create remainders
baremaining = batotal - baT10.DV_Incident_Frequency.sum() # 25987
bcremaining = bctotal - bcT10.DV_Incident_Frequency.sum() # 19934
bcpopremaining = by_county.Pop_Est_2019.sum() - bcT10.Pop_Est_2019.sum()
# adding Other row
baDict = pd.DataFrame({'Agency_Name':'Other', 'Agency_Type':'Remaining', 'DV_Incident_Frequency': baremaining}, index=[11])
bcDict = pd.DataFrame({'County':'Other', 'DV_Incident_Frequency':bcremaining, 'Pop_Est_2019':bcpopremaining}, index=[11])
baT10 = pd.concat([baT10, baDict], axis=0, ignore_index=False)
bcT10 = pd.concat([bcT10, bcDict], axis=0, ignore_index=False)


In [ ]:
# by_agency DV distribution pie chart
by_agency_fig = px.pie(baT10, values='DV_Incident_Frequency', names='Agency_Name', color_discrete_sequence=px.colors.qualitative.Dark24)
by_agency_fig.update_layout(title_text='DV Distribution - Top 10 Agencies')
by_agency_fig.show()
# by_county DV distribution pie chart
by_county_fig = px.pie(bcT10, values='DV_Incident_Frequency', names='County', color_discrete_sequence=px.colors.qualitative.Dark24)
by_county_fig.update_layout(title_text='DV Distribution - Top 10 Counties')
by_county_fig.show()
# by_county Pop distribution pie chart
by_county_pop_fig = px.pie(bcT10, values='Pop_Est_2019', names='County', color_discrete_sequence=px.colors.qualitative.Dark24)
by_county_pop_fig.update_layout(title_text='Pop Distribution - Top 10 Counties')
by_county_pop_fig.show()

# write to image
by_agency_fig.write_image('../Visuals/Mart2/DV_Distr_by_Agency.png', engine='kaleido')
by_county_fig.write_image('../Visuals/Mart2/DV_Distr_by_County.png', engine='kaleido')
by_county_pop_fig.write_image('../Visuals/Mart2/Pop_Distr_by_County.png', engine='kaleido')

In [350]:
print(len(by_agency))
by_agency.head()

359


,Agency_Name,Agency_Type,DV_Incident_Frequency,Office Total,Civilian Total
0,Adamsville Police Department,City,5,7,1
1,Alamo Police Department,City,39,4,0
2,Alcoa Police Department,City,139,46,8
3,Alexandria Police Department,City,3,3,1
4,Algood Police Department,City,28,15,0


In [356]:
#agency_stat = pd.read_excel('../Data/Mart2/2020 Employee Counts for Dr Stickle.xlsx')
agency_stat.head()

,agyori,AgyName,rptyear,Officer Male,Officer Female,Officer Total,Civilian Male,Civilian Female,Civilian Total,Law Enforcement - Male - Total,Law Enforcement - Female - Total,Law Enforcement Total,Agency_Type,Agency_DV_Rate
0,TN0310000,Grundy County Sheriff's Office,2020,15,3,18,13,10,23,28,13,41,County,135.0
1,TN0250100,Jamestown Police Department,2020,9,0,9,0,0,0,9,0,9,City,15.0
2,TN0260300,Cowan Police Department,2020,3,0,3,0,0,0,3,0,3,City,13.0
3,TN0740200,Greenbrier Police Department,2020,14,1,15,0,1,1,14,2,16,City,43.0
4,TN0790500,University of Tennessee Health Sciences Center,2020,24,8,32,16,16,32,40,24,64,College/Univerity,6.0


In [334]:
for i in range(len(agency_stat)):
    oldname = agency_stat.iloc[i].AgyName
    if 'Sheriffs' in oldname:
        newname = "Sheriff's".join(oldname.split('Sheriffs'))
        agency_stat.loc[agency_stat.AgyName == oldname, 'AgyName'] = newname

individual_checks = {
    'University of Tennessee Health Science C':'University of Tennessee Health Sciences Center',
    'King University - Security Department':'King University',
    'Tennessee Tech University':'Tennessee Technological University',
    'Southwest Tennessee Community College Po':'Southwest Tennessee Community College',
    'Fort Pillow Historic State Park':'Fort Pillow State Historic Park',
    'Memphis Shelby County Airport Authority ':'Memphis Shelby County Airport Authority Police',
    'Metro Hartsville/Trousdale County Sherif':"Metro Hartsville/Trousdale County Sheriff's Office",
    'Metro Nashville Park Police':'Nashville Metro Park Police',
     "Perry County Sheriff's Department":"Perry County Sheriff's Office",
    "South Carthage Police Department":"South  Carthage Police Department",
    "Tri-Cities Airport Public Safety Departm":'Tri-Cities Airport Public Safety Department',
    "Vonore Police Department":'Vonore City Police Department',
    "Warriors Path State Park":"Warriors' Path State Park",
    "Lafayette Police Department":"LaFayette Police Department",
    
}

for i in range(len(agency_stat)):
    oldname = agency_stat.iloc[i].AgyName
    if 'Tims' in oldname:
        newname = "Tim's".join(oldname.split('Tims'))
        agency_stat.loc[agency_stat.AgyName == oldname, 'AgyName'] = newname
    elif oldname in list(individual_checks.keys()):
        agency_stat.loc[agency_stat.AgyName == oldname, 'AgyName'] = individual_checks[oldname]


In [349]:
for i in range(len(by_agency)):
    name = by_agency.iloc[i].Agency_Name
    stat_row = agency_stat.loc[agency_stat.AgyName == name]
    try:
        off_total = stat_row['Officer Total'].values[0]
        civ_total = stat_row['Civilian Total'].values[0]
        by_agency.loc[by_agency.Agency_Name == name, 'Office Total'] = int(off_total)
        by_agency.loc[by_agency.Agency_Name == name, 'Civilian Total'] = int(civ_total)
    except IndexError:
        print(name)
        by_agency.loc[by_agency.Agency_Name == name, 'Office Total'] = ''
        by_agency.loc[by_agency.Agency_Name == name, 'Civilian Total'] = ''

Bethel University
Fisk University
Lane College
Lee University
Rhodes College
Union University


In [355]:
# adding DV_Rates_by_Agency to stats table, basically the reverse of above
for i in range(len(by_agency)):
    by_ag_row = by_agency.iloc[i]
    name = by_ag_row.Agency_Name
    type = by_ag_row.Agency_Type
    rate = by_ag_row.DV_Incident_Frequency
    try:
        agency_stat.loc[agency_stat.AgyName == name, 'Agency_Type'] = type
        agency_stat.loc[agency_stat.AgyName == name, 'Agency_DV_Rate'] = int(rate)
    except IndexError:
        print(name)
        agency_stat.loc[agency_stat.AgyName == name, 'Agency_Type'] = ''
        agency_stat.loc[agency_stat.AgyName == name, 'Agency_DV_Rate'] = ''


In [362]:
# checking before exporting agency_stat
for i in range(len(agency_stat)):
    stat_row = agency_stat.iloc[i]
    name = stat_row.AgyName
    try:
        rate = int(stat_row.Agency_DV_Rate)
    except ValueError:
        continue
    # compare
    old_rate = by_agency.loc[by_agency.Agency_Name == name].DV_Incident_Frequency.values[0]
    if old_rate != rate:
        print(name, old_rate, rate)

In [363]:
agency_stat.to_excel('../Tables/Mart2/Agency_Employee_and_DVRates.xlsx')

In [335]:
# from 113 to 21 with sheriff's fix, w/ independent fixes 21 to 6
miss = []
df_names = list(by_agency.Agency_Name)
ex_names = list(agency_stat.AgyName)
for name in df_names:
    if name not in ex_names:
        miss.append(name)
len(miss)

6

In [338]:
true_miss = {}
for name in miss:
    true_miss[name] = 'missing info'

In [337]:
# implement Sheriffs ot Sheriff's fix;
i = 0
# for name in miss:
#     if 'Sheriff' in name:
#         i+=1
print(i)

for name in ex_names:
    if 'University of Tennessee' in name:
        print(f'-{name}-')

SyntaxError: invalid syntax (2824084972.py, line 8)

In [ ]:
t20_a = by_agency.sort_values('DV_Incident_Frequency', ascending=False)[:20]
t20_a.index = np.arange(1,21)


In [ ]:
# t20_a.reset_index(drop=False).plot.bar(x='index', y='DV_Incident_Frequency', xlabel='Ranked Agency')

In [ ]:
# checking indices line up
inc_freq = list(by_county.sort_values('DV_Incident_Frequency', ascending=False)[:20].index)
off_freq = list(by_county.sort_values('DV_Offense_Frequency', ascending=False)[:20].index)
print('Incident Ranking | Offense Ranking')
for i in range(20):
    print(inc_freq[i], '- - -',off_freq[i], inc_freq[i] == off_freq[i])

In [ ]:
t20 = by_county.sort_values('DV_Incident_Frequency', ascending=False)[:20]
t20.index = np.arange(1, 21)
t20.head()

In [ ]:
# t20.reset_index(drop=False).plot.bar(x='index', y='DV_Incident_Frequency', xlabel='Ranked County')

In [351]:
by_agency.to_csv('../Tables/Mart2/DV_Rates_by_agency.csv')

In [ ]:
# # Checking matching index
# for i in range(len(by_county)):
#     bc_county = by_county.iloc[i].County
#     c_county = census.iloc[i].County
#     if bc_county != c_county:
#         print(i, bc_county, c_county)

In [ ]:
# Features: 
# -City, ignored for time, might try a partial 
# -County, other
# -Agency Name, other
# -Pri Location, locat
# -Pop of City / County, investigate this...
# -Rates per 1k, 10k, 100k residents
# -Weigth (essentially how many offenses were reported in this incident), other

In [ ]:
# hunting for City
broken = [] # includes missing and broken addresses
for i in range(len(admin_df)):
    row = admin_df.iloc[i]
    st = row.Address
    # checking if 'TN' is second to last entry on all of them
    try:
        check = st.split(',')[-2]
    except (AttributeError, IndexError):
        key = row.Incident_Key
        broken.append(key)
        continue

    if 'TN' not in check:
        key = row.Incident_Key
        broken.append(key)

len(broken)

In [ ]:
# 369 have ",,,,," as address
# 918 isnull() is True
# 47 Can have cities/Towns extracted manually
i = 0
for key in broken:
    X = admin_df.loc[admin_df.Incident_Key == key]
    if X.Address.values[0] != ',,,,,' and type(X.Address.values[0]) == str: 
        try:
            place = X.Address.values[0].split(',')[-3]
            if place != '':
                i+=1 
                print(key, place)
        except IndexError:
            continue
print(i)

key_city = {
    'TN0010000<>2001100069<>01/10/20': 'POWELL',
    'TN0380000<>2020-000796<>01/18/20': 'BROWNSVILLE',
    'TN0510000<>SH20032026<>01/23/20': 'HOHENWALD',
    'TN0600100<>2020-000968<>02/21/20':'COLUMBIA',
    'TN0600100<>2020-001108<>02/27/20': 'COLUMBIA',
    'TN0740100<>2020-000316<>02/10/20': 'SPRINGFIELD',
    'TN0470000<>2001030154<>01/03/20': 'KNOXVILLE',
    'TN0610000<>202002499<>03/31/20': 'TEN MILE',
    'TN0380000<>2020-003189<>03/09/20': 'BROWNSVILLE',
    'TN0600100<>2020-001258<>03/05/20': 'COLUMBIA',
    'TN0470000<>2002291923<>02/23/20': 'HEISKELL',
    'TN0470000<>2002291940<>02/29/20': 'POWELL',
    'TN0470000<>2004060197<>04/06/20': 'MASCOT',
    'TN0470000<>2004080311<>04/08/20': 'POWELL',
    'TN0440000<>030320-13288<>03/03/20': 'GAINESBORO',
    'TN0470000<>2002281885<>02/28/20': 'POWELL',
    'TN0470000<>2004130549<>04/13/20': 'KNOXVILLE',
    'TN0740000<>2020-001070<>04/10/20': 'CEDAR HILL',
    'TN0410000<>200503-1652<>05/03/20': 'LYLES',
    'TN0440000<>052120-13385<>05/21/20': 'GAINESBORO',
    'TN0950100<>2020025898<>06/24/20': 'Lebanon',
    'TN0440000<>060820-13406<>06/08/20': 'GAINESBORO',
    'TN0290300<>2006280309<>06/28/20': 'BEAN STATION',
    'TN0290300<>2007110171<>07/11/20': 'BEAN STATION',
    'TN0470000<>2007060390<>07/06/20': 'STRAWBERRY PLAINS',
    'TN0470000<>2007120722<>07/12/20': 'KNOX',
    'TN0470000<>2007120717<>07/12/20': 'CORRYTON',
    'TN0470000<>2006100679<>06/10/20:': 'KNOX',
    'TN0470000<>2007181112<>07/18/20': 'KNOX',
    'TN0860000<>2007002253<>07/26/20': 'UNICOI',
    'TN0170000<>2002151817<>02/15/20': 'Miami',
    'TN0530000<>20200730871<>07/16/20': 'LENOIR CITY',
    'TN0020100<>2007100050<>07/10/20': 'SHELBYVILLE',
    'TN0470000<>2008241521<>08/24/20': 'KNOX',
    'TN0610000<>202000043<>01/02/20': 'DECATUR',
    'TN0470000<>2008301940<>08/30/20': 'POWELL',
    'TN0440000<>080720-13540<>08/07/20': 'COOKEVILLE',
    'TN0470000<>2008301972<>08/30/20': 'CORRYTON',
    'TN0170000<>2008122130<>08/12/20': 'Gum Flats',
    'TN0020100<>2009190034<>09/19/20': 'SHELBYVILLE',
    'TN0540100<>202000000000<>09/28/20': 'Athens', 
    'TN0470000<>2010060450<>10/06/20': 'POWELL',
    'TN0470000<>2008160966<>08/16/20': 'POWELL',
    'TN0440000<>090520-13613<>09/05/20': 'COOKEVILLE',
    'TN0190100<>20200588121<>09/12/20': 'Nashville',
    'TN0410000<>201022-1156<>10/22/20': 'NUNNELLY',
    'TN0470000<>2011141072<>11/14/20': 'MASCOT',
    'TN0470000<>2011201575<>11/20/20': 'STRAWBERRY PLAINS',
    'TN0470000<>2011241935<>11/24/20': 'KNOX',
    'TN0830100<>GLTN20-04602<>11/03/20': 'GALLATIN',
    'TN0470000<>2012241929<>12/24/20': 'KNOXVILLE',
    'TN0440000<>011620-13127<>01/16/20': 'GAINESBORO',
    'TN0440000<>072020-13497<>07/19/20': 'GAINESBORO',
    'TN0470000<>2012080552<>12/08/20': 'STRAWBERRY PLAINS',
    'TN0470000<>2012120991<>12/12/20': 'CORRYTON',
    'TN0470000<>2102221624<>09/01/20': 'POWELL',
    'TN0470000<>2007181150<>07/18/20': 'KNOX',
    'TN0640000<>2020-0436<>12/12/20': 'FAYETTEVILLE',
    'TN0410000<>201031-1322<>10/31/20': 'NUNNELLY',
    'TN0170000<>2009041123<>09/04/20': 'Paragould',
    'TN0190100<>20200414761<>06/19/20': 'ANTIOCH',
    'TN0610000<>202005168<>07/01/20': 'BIRCHWOOD',
    'TN0740100<>2020-001624<>07/11/20': 'SPRINGFIELD',
    'TN0330200<>20-017126<>12/30/20': 'EAST RIDGE',
}